In [25]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [26]:
# read picture as GRAYSCALE
img=cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [27]:
# binary image (white->1 black->0)
def binarize(image):
    answer = np.zeros(image.shape, dtype=np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i][j]>127:
                answer[i][j] = 1
    return answer

In [28]:
# ture binary image to whith-black image
def binary2WB(binary_image):
    WB_image = np.zeros(binary_image.shape,dtype = np.uint8)
    
    for i in range(binary_image.shape[0]):
        for j in range(binary_image.shape[1]):
            if binary_image[i][j]==1:
                WB_image[i][j] = 255
    
    return WB_image

In [29]:
# down sampleing(512*512 -> 64*64)
def down_sample(binary_image):
    answer = np.zeros((64,64), dtype=np.uint8)
    for i in range(64):
        for j in range(64):
            if binary_image[i*8][j*8]==1:
                answer[i][j] = 1
                
    return answer

In [30]:
# decide the corner type
def Yokoi_corner_type(b,c,d,e):
    if b==c and (d!=b or e!=b):
        return 'q'
    elif b==c and d==b and e==b:
        return 'r'
    elif b!=c:
        return 's'
    else:
        print("error! not a corner type!")

In [31]:
# decide the Yokoi number
def Yokoi_number(sub_image):
    dictionary = {'q': 0, 'r': 0, 's': 0}
    
    # top_right
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[1][2],sub_image[0][2],sub_image[0][1])]+=1
    # top_left
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[0][1],sub_image[0][0],sub_image[1][0])]+=1
    # buttom_left
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[1][0],sub_image[2][0],sub_image[2][1])]+=1
    # buttom_right
    dictionary[Yokoi_corner_type(sub_image[1][1],sub_image[2][1],sub_image[2][2],sub_image[1][2])]+=1
    
    if dictionary['s'] == 4:
        return 0
    elif dictionary['q'] == 1:
        return 1
    elif dictionary['q'] == 2:
        return 2
    elif dictionary['q'] == 3:
        return 3
    elif dictionary['q'] == 4:
        return 4
    elif dictionary['r'] == 4:
        return 5
    else:
        print("error! not a yokoi number!")
        

In [32]:
def Yokoi(binary_image):
    
    result_shape_0 = binary_image.shape[0]
    result_shape_1 = binary_image.shape[1]
    
    zero_padding_binary_image = np.zeros((result_shape_0 + 2,result_shape_1 + 2),dtype=int)
    zero_padding_binary_image[1:result_shape_0 + 1,1:result_shape_1 + 1] = binary_image[0:result_shape_0][0:result_shape_0]
    
    return_matrix = np.zeros((result_shape_0,result_shape_1),dtype=int)
    return_matrix.fill(-1)
    
    for i in range(1,result_shape_0 + 1):
        for j in range(1,result_shape_1 + 1):
            if zero_padding_binary_image[i][j] == 1:
                return_matrix[i-1][j-1] =  Yokoi_number(zero_padding_binary_image[i-1:i+2,j-1:j+2])
    
    return return_matrix

In [33]:
# q -> 0   not interesting pixel
# p -> 1   target pixel
def Pair_operation(binary_image,yokoi_matrix):
    checking_list = [[0,1],[-1,0],[0,-1],[1,0]]
    
    padding_yokoi = np.ones((binary_image.shape[0]+2,binary_image.shape[1]+2),dtype=int)
    padding_yokoi = -1*padding_yokoi
    padding_yokoi[1:padding_yokoi.shape[0]-1,1:padding_yokoi.shape[1]-1] = yokoi_matrix[0:binary_image.shape[0],0:binary_image.shape[1]]
    
    
    result = np.zeros(binary_image.shape,dtype=np.uint8)
    for i in range(1,padding_yokoi.shape[0]-1):
        for j in range(1,padding_yokoi.shape[1]-1):
            if binary_image[i-1][j-1]==1 and padding_yokoi[i][j]==1:
                for k in range(4):
                    check_point_i = i + checking_list[k][0]
                    check_point_j = j + checking_list[k][1]
                    
                    if padding_yokoi[check_point_i][check_point_j]==1:
                        result[i-1][j-1] = 1
                        break
                        
    return result

In [34]:
def Connected_Shrink_Operator_with_Pair(pair_image,binary_image):
    result = np.zeros(binary_image.shape,dtype = int)
    
    zero_padding_binary_image = np.zeros((binary_image.shape[0] + 2,binary_image.shape[1] + 2),dtype='int')
    zero_padding_binary_image[1:binary_image.shape[0] + 1,1:binary_image.shape[1] + 1] = binary_image[0:binary_image.shape[0],0:binary_image.shape[1]]
    
    for i in range(1,binary_image.shape[0] + 1):
        for j in range(1,binary_image.shape[1] + 1):
            # it is white pixel
            if zero_padding_binary_image[i][j] == 1:
                yokoi_result_i_j =  Yokoi_number(zero_padding_binary_image[i-1:i+2,j-1:j+2])
                if pair_image[i-1][j-1] == 1 and yokoi_result_i_j == 1:
                    zero_padding_binary_image[i][j] = 0
    
    return  zero_padding_binary_image[1:binary_image.shape[0]+1,1:binary_image.shape[1]+1]

In [35]:
def Thining(binary_image):

    yokoi_matrix = Yokoi(binary_image)    
    pair_image = Pair_operation(binary_image,yokoi_matrix)
    thining = Connected_Shrink_Operator_with_Pair(pair_image,binary_image)
    
    return thining

In [36]:
binary_image = binarize(img)
sampled_binary_image = down_sample(binary_image)


thining = Thining(sampled_binary_image)
for i in range(6):
    thining = Thining(thining)
    
result = binary2WB(thining)
cv2.imwrite('HW7.bmp', result)

# draw a bigger one to check the answer
result_enlarge = np.zeros((result.shape[0]*8,result.shape[1]*8),dtype=np.uint8)

for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        if result[i][j]==255:
            result_enlarge[i*8:i*8+8,j*8:j*8+8] = 255

cv2.imwrite('HW7_enlarge.bmp', result_enlarge)


True